### Projeto simples de um sostema bancario 

--> funções principais:
    Sacar com limite de 3 saques diarios, cada saque de no máximo 500 reais
    depositar 
    Extrato --> com todas movimentações.

In [2]:
import datetime

# --- Variáveis Globais ---
saldo = 0
limite_saque = 500
extrato = []
numero_transacoes = 0
LIMITE_TRANSACOES = 10 # Novo limite de 10 transações diárias

# --- Funções ---

def depositar(valor):
    global saldo, extrato, numero_transacoes

    # Verifica se o limite de transações foi atingido
    if numero_transacoes >= LIMITE_TRANSACOES:
        print("❌ Erro: Limite diário de 10 transações foi atingido.")
        return # Interrompe a função

    if valor > 0:
        saldo += valor
        numero_transacoes += 1
        # Adiciona a data e hora da transação ao extrato
        timestamp = datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")
        extrato.append(f"[{timestamp}] Depósito: +R$ {valor:.2f}")
        print(f"✅ Depósito de R$ {valor:.2f} realizado com sucesso.")
    else:
        print("❌ Erro: Valor inválido para depósito.")

def sacar(valor):
    global saldo, extrato, numero_transacoes, limite_saque

    # Verifica se o limite de transações foi atingido
    if numero_transacoes >= LIMITE_TRANSACOES:
        print("❌ Erro: Limite diário de 10 transações foi atingido.")
        return

    # Demais validações
    excedeu_saldo = valor > saldo
    excedeu_limite = valor > limite_saque
    valor_invalido = valor <= 0

    if excedeu_saldo:
        print("❌ Erro: Saldo insuficiente.")
    elif excedeu_limite:
        print(f"❌ Erro: Limite por saque é de R$ {limite_saque:.2f}.")
    elif valor_invalido:
        print("❌ Erro: Valor inválido para saque.")
    else:
        saldo -= valor
        numero_transacoes += 1
        # Adiciona a data e hora da transação ao extrato
        timestamp = datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")
        extrato.append(f"[{timestamp}] Saque:    -R$ {valor:.2f}")
        print(f"✅ Saque de R$ {valor:.2f} realizado com sucesso.")

def mostrar_extrato():
    print("\n" + "📄 EXTRATO BANCÁRIO ".center(40, "="))
    if not extrato:
        print("Nenhuma movimentação realizada.")
    else:
        for operacao in extrato:
            print(operacao)
    print("=" * 40)
    print(f"Saldo atual: R$ {saldo:.2f}")
    print(f"Transações realizadas hoje: {numero_transacoes}/{LIMITE_TRANSACOES}")
    print("=" * 40)


# --- Exemplo de Uso ---
print("Bem-vindo ao Banco DIO PRO!")

depositar(2000)
sacar(300)
sacar(500)
mostrar_extrato()

# Testando o limite de transações
print("\n--- Testando o limite de transações ---")
for i in range(8): # Realiza mais 8 transações para atingir o limite de 10
    sacar(50)

# Esta transação deve ser bloqueada
depositar(100)
sacar(50)

mostrar_extrato()

Bem-vindo ao Banco DIO PRO!
✅ Depósito de R$ 2000.00 realizado com sucesso.
✅ Saque de R$ 300.00 realizado com sucesso.
✅ Saque de R$ 500.00 realizado com sucesso.

==========📄 EXTRATO BANCÁRIO ===========
[05-08-2025 13:38:20] Depósito: +R$ 2000.00
[05-08-2025 13:38:20] Saque:    -R$ 300.00
[05-08-2025 13:38:20] Saque:    -R$ 500.00
Saldo atual: R$ 1200.00
Transações realizadas hoje: 3/10

--- Testando o limite de transações ---
✅ Saque de R$ 50.00 realizado com sucesso.
✅ Saque de R$ 50.00 realizado com sucesso.
✅ Saque de R$ 50.00 realizado com sucesso.
✅ Saque de R$ 50.00 realizado com sucesso.
✅ Saque de R$ 50.00 realizado com sucesso.
✅ Saque de R$ 50.00 realizado com sucesso.
✅ Saque de R$ 50.00 realizado com sucesso.
❌ Erro: Limite diário de 10 transações foi atingido.
❌ Erro: Limite diário de 10 transações foi atingido.
❌ Erro: Limite diário de 10 transações foi atingido.

==========📄 EXTRATO BANCÁRIO ===========
[05-08-2025 13:38:20] Depósito: +R$ 2000.00
[05-08-2025 13:38:20